# Kryptools

Warning: These tools are intended for experimenting with cryptographic algorithms. They are not intended for use in real applications.

## Zmod: Ring of integers modulo n

To define $\mathbb{Z}_5$, the [ring of integers modulo the prime $5$](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n) (in this case a [Galois field](https://en.wikipedia.org/wiki/Finite_field)), use

In [1]:
from kryptools import Zmod

Z_5 = Zmod(5)

To declare $3$ as an element of $\mathbb{Z}_5$ use

In [2]:
Z_5(3)

3

Basic arithmetic operations work as expected:

In [3]:
Z_5(3) + Z_5(2)

0

In [4]:
Z_5(3) ** -1

2

To determine the multiplicative [order](https://en.wikipedia.org/wiki/Order_(group_theory)) of an element in the group $\mathbb{Z}_n^*$ use:

In [5]:
Z_5(3).order()

4

You can also check if an element is a [generator](https://en.wikipedia.org/wiki/Primitive_root_modulo_n) (a.k.a primitive root) of $\mathbb{Z}_n^*$:

In [6]:
Z_5(3).is_generator()

True

You can get one generator or all generators

In [7]:
Z_5.generator(), list(Z_5.generators())

(2, [2, 3])

as well as all elements of $\mathbb{Z}_n^*$

In [8]:
list(Z_5.star())

[1, 2, 3, 4]

To test if $\mathbb{Z}_n^*$ is cyclic use

In [9]:
Z_5.is_cyclic()

True

For [Learning With Errors](https://en.wikipedia.org/wiki/Learning_with_errors) a symmetric representative is frequently required:

In [10]:
[a.sharp() for a in Z_5]

[0, 1, 2, -2, -1]

The absolute value will be the absolute value of the symmetric representative:

In [11]:
[abs(a) for a in Z_5]

[0, 1, 2, 2, 1]

If you set the option `short` to `False` then $\pmod{5}$ will be added:

In [12]:
Z_5.short = False
Z_5(6)

1 (mod 5)

## Galois field $\mathrm{GF}(2^n)$

There is also some support for playing with the [Galois field](https://en.wikipedia.org/wiki/Finite_field) $\mathrm{GF}(2^n)$. Elements are specified by integers from $0$ to $2^n-1$, which are identified with polynomials whose coefficients are the binary digits of the integer. Multiplication is polynomial multiplication modulo a given irreducible polynomial.
If the modulus has the $n$'th bit set, the lowest bit corresponds to the constant coefficient (like in AES). Otherwise, the bit order is reversed (like for Ghash).
The default polynomial is a [Conway polynomial](https://en.wikipedia.org/wiki/Conway_polynomial_(finite_fields)) from [Frank Lübeck’s list](https://www.math.rwth-aachen.de/~Frank.Luebeck/data/ConwayPol/index.html).

To define the Galois field GF(2^8) for [AES](https://en.wikipedia.org/wiki/Rijndael_MixColumns) with polynomial $1 + x + x^3 + x^4 + x^8$ use (this will create a class `types.GF2^8`, which will be the return value)

In [13]:
from kryptools import GF2

gf = GF2(8, modulus=0b100011011)  # x^8 + x^4 + x^3 + x + 1 = AES

Note: If you do not specify the modulus, the corresponding Conway polynomial $1 + x^2 + x^3 + x^4 + x^8$ will be used.

To declare $3$ as an element of our Galois field use (elements are displayed as hex numbers)

In [14]:
gf(3)

03

The usual arithmetic operations are supported.

In [15]:
gf(2) * gf(135)

15

Elements from $\mathrm{GF}(2^n)$ can be convertet to `int`, `bytes`, or to a polynomial (see the `Poly` class below):

In [16]:
_.poly()

x^4 + x^2 + 1

For convenience, the AES Galois field is also available as `GF2_aes`. It has an additional method for applying the AES sbox (to get the inverse sbox, use the option `inv=True`):

In [17]:
from kryptools import GF2_aes

GF2_aes(0).sbox()

63

### Showcase: Ghash

For [Ghash](https://en.wikipedia.org/wiki/Galois/Counter_Mode) the polynomial is $1 + x + x^2 + x^7 + x^{128}$. You can explicitely define the corresponding Galois filed using

In [18]:
gf = GF2(128, modulus=0b11100001 << 120)  # 1 + x + x^2 + x^7 (+ x^128) = Ghash

For convenience, it is also avaiable as `GF2_ghash`:

In [19]:
from kryptools import GF2_ghash


def ghash(data: bytes, h: bytes) -> bytes:
    "Compute ghash of a bytestring `data` with a given key `h`."
    # First pad data such that it is a multiple of the block length 16*8=128
    pad = len(data) % 16
    if pad > 0:
        data += b"\x00" * (16 - pad)
    # Now compute ghash recursively
    h = GF2_ghash(h)
    g = GF2_ghash(0)
    for i in range(len(data) // 16):
        g = h * (g + GF2_ghash(data[i * 16 : (i + 1) * 16]))
    return bytes(g)


h = b"7\xa0\x8e\xd3\x00\x93\xa13\xb1\xbbJ\xe0\xb8\xf3`\x1f"  # 128 bit key
text = b"A boring message"

ghash(text, h)

b'\xcc\x16\x99\x1d\x8d`\xadu\xc9+\xe1,\x98\x13j\x17'

## Primes

A tuple of [primes](https://en.wikipedia.org/wiki/Prime_number) below a bound $B$ can be obtained using the [sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes):

In [20]:
from kryptools import sieve_eratosthenes

sieve_eratosthenes(20)

(2, 3, 5, 7, 11, 13, 17, 19)

The Prime-counting function $\pi(x)$ and the primorial $\#x$ is implemented by counting the number of primes returned by the sieve of Eratosthenes:

In [21]:
from kryptools import prime_pi, primorial

prime_pi(20), primorial(7)

(8, 210)

To test if a number is probable prime you can use

In [22]:
from kryptools import is_prime

is_prime(primorial(13) + 1)

False

For numbers below $3317044064679887385961981$ it performs a deterministic variant of the [Miller-Rabin test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test)

In [23]:
from kryptools import miller_rabin_test

miller_rabin_test(5459, [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41])

False

Otherwise it combines trial division with small primes (for speed), a Miller-Rabin test with base $2$, and a [strong Lucas test](https://en.wikipedia.org/wiki/Lucas_pseudoprime#Implementing_a_Lucas_probable_prime_test) ([Baillie–PSW primality test](https://en.wikipedia.org/wiki/Baillie%E2%80%93PSW_primality_test)).

In [24]:
from kryptools import lucas_test

lucas_test(5459)

True

The next/previous prime larger/smaller or equal to a given number

In [25]:
from kryptools import next_prime, previous_prime

next_prime(100), previous_prime(100)

(101, 97)

A pseudorandom prime (based on `secure.randbits`, so hopefully cryptographically safe) of given bit length

In [26]:
from kryptools import random_prime

p = random_prime(128)
assert is_prime(p)
assert p.bit_length() == 128

Similarly, a pseudorandom [safe prime](https://en.wikipedia.org/wiki/Safe_and_Sophie_Germain_primes) $p$ of given bit length with $2$ generating the large subgroup, $\text{ord}(2)=(p-1)/2$:

In [27]:
from kryptools import is_safeprime, order, random_safeprime

p = random_safeprime(128)
assert is_safeprime(p)
assert p.bit_length() == 128
assert order(2, p) == (p - 1) // 2

A list of the first safe primes together with the corresponding Sophie Germain primes can be optained as follows:

In [28]:
[((p - 1) // 2, p) for p in sieve_eratosthenes(200) if is_safeprime(p)]

[(2, 5),
 (3, 7),
 (5, 11),
 (11, 23),
 (23, 47),
 (29, 59),
 (41, 83),
 (53, 107),
 (83, 167),
 (89, 179)]

Finally, a pseudorandom [strong prime](https://en.wikipedia.org/wiki/Strong_prime) $p$ with factors of $p\pm 1$ of given bit length using Gordon's algorithm:

In [29]:
from kryptools import random_strongprime

t, s, r, p = random_strongprime(128)
assert is_prime(p) and is_prime(t) and is_prime(s) and is_prime(r)
assert (p + 1) % s == 0 and (p - 1) % r == 0 and (r - 1) % t == 0
p.bit_length()

271

## Number theory

To perform the [extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) use

In [30]:
from kryptools.nt import egcd

a, b = 7, 8
g, x, y = egcd(a, b)

assert x * a + y * b == g
g, x, y

(1, -1, 1)

To solve a list of congruences using the [Chinese Remainder Theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem) use

In [31]:
from kryptools import crt

a, m = [1, 2, 3], [2, 3, 5]
sol = crt(a, m)

assert [sol % p for p in m] == a
sol

23

The moduli are assumed to be coprime. If not, you can use:

In [32]:
m = [4, 8, 12, 30]
x = 45
a = [x % mi for mi in m]

assert crt(a, m, coprime=False) == x

To compute the [continued fraction](https://en.wikipedia.org/wiki/Continued_fraction) expansion of a rational number use

In [33]:
from fractions import Fraction

Fraction.__repr__ = Fraction.__str__
from kryptools import cf, convergents

cf(Fraction(18, 23))

[0, 1, 3, 1, 1, 2]

To get the convergents use

In [34]:
convergents(_)

[0, 1, 3/4, 4/5, 7/9, 18/23]

To compute the [Jacobi symbol](https://en.wikipedia.org/wiki/Jacobi_symbol) use

In [35]:
from kryptools import jacobi_symbol

jacobi_symbol(4, 7)

1

Since it equals $+1$, the number is a [quadratic residue](https://en.wikipedia.org/wiki/Quadratic_residue) and has a square root modulo the prime $7$

In [36]:
from kryptools import sqrt_mod

sqrt_mod(4, 7)

2

To compute [Euler's $\varphi$ function](https://en.wikipedia.org/wiki/Euler%27s_totient_function) (a.k.a totient function) or [Carmichael's $\lambda$ function](https://en.wikipedia.org/wiki/Carmichael_function) (a.k.a reduced totient function) or [Möbius' $\mu$ function](https://en.wikipedia.org/wiki/Moebius_function) use

In [37]:
from kryptools import carmichael_lambda, euler_phi, moebius_mu

n = 100
euler_phi(n), carmichael_lambda(n), moebius_mu(n)

(40, 20, 0)

A test if a number is a [Carmichael number](https://en.wikipedia.org/wiki/Carmichael_number):

In [38]:
from kryptools import is_carmichael_number

is_carmichael_number(561)

True

To compute the order of an element of the [multiplicative group of integers modulo $n$](https://en.wikipedia.org/wiki/Multiplicative_group_of_integers_modulo_n), $\mathbb{Z}_n^*$, use

In [39]:
from kryptools.nt import order

order(5, 7)

6

### Showcase: Asmuth-Bloom scheme

Using the Chinise Remainder Theorem you can share a secret $s\in\mathbb{Z}_m$ among $n$ persons, such that $r$ persons can recover the secret, using the [Asmuth-Bloom scheme](https://en.wikipedia.org/wiki/Secret_sharing_using_the_Chinese_remainder_theorem#Asmuth%E2%80%93Bloom_threshold_secret_sharing_scheme). First we need to generate suitable parameters. We choose primes for the moduli to ensure that they are coprime.

In [40]:
from math import prod

from kryptools import crt, next_prime

n, r = 5, 3
m = 2**32  # max size of the secret

M = []
mm = 2 * m
for _ in range(n):
    mm = next_prime(mm + 1)
    M.append(mm)

M1 = prod(M[:r])
assert m * prod(M[1 - r :]) < M1  # Asmuth-Bloom condition
M

[8589934609, 8589934621, 8589934627, 8589934631, 8589934651]

Now we can split a secret $s$ into $n$ pieces:

In [41]:
from random import randint, seed

seed(0)

s = randint(0, m - 1)
t = randint(1, (M1 - s) // m)
x = s + t * m
del t  # t is no longer needed and should be deleted for security reasons
assert x < M1

A = [x % mm for mm in M]
A

[953188841, 2267150034, 7376225206, 341660855, 3090211041]

Let us check that it worked:

In [42]:
from random import sample

smpl = sample(range(n), r)

assert crt([A[i] for i in smpl], [M[i] for i in smpl]) % m == s

## Factoring integers

To factor an integer $n$ into its prime factors use

In [43]:
from kryptools import factorint

n = 2**128 + 1
factorint(n, verbose=1)

Factoring: 340282366920938463463374607431768211457 (39 digits)
Trial division found: []
Round 1 (B1=5000)
Factors found (ecm):  [59649589127497217, 5704689200685129054721]


{59649589127497217: 1, 5704689200685129054721: 1}

There are also tests if a number is a [perfect power](https://en.wikipedia.org/wiki/Perfect_power) (if it is not, `None` will be returned)

In [44]:
from kryptools import perfect_power

perfect_power(100)

(10, 2)

or a prime power (if it is not, `None` will be returned)

In [45]:
from kryptools import prime_power

prime_power(9973**100)

(9973, 100)

For factoring you can also use a specific algorithm (if no factor is found, `None` is returned):

[Fermat's method](https://en.wikipedia.org/wiki/Fermat%27s_factorization_method)

In [46]:
from kryptools.factor_fmt import factor_fermat

factor_fermat(832283)

[487, 1709]

[Pollard $p-1$](https://en.wikipedia.org/wiki/Pollard%27s_p_%E2%88%92_1_algorithm)

In [47]:
from kryptools.factor_pm1 import factor_pm1

factor_pm1(832283)

1709

[Pollard $\rho$](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm)

In [48]:
from kryptools.factor_rho import factor_rho

factor_rho(832283)

487

[Lentstra's ECM](https://en.wikipedia.org/wiki/Lenstra_elliptic-curve_factorization)

In [49]:
from kryptools.factor_ecm import factor_ecm

factor_ecm(832283)

487

A variant of [Dixon's method](https://en.wikipedia.org/wiki/Dixon%27s_factorization_method)

In [50]:
from kryptools.factor_dix import factor_dixon

factor_dixon(832283)

1709

[Quadratic sieve](https://en.wikipedia.org/wiki/Quadratic_sieve) (with a single polynomial)

In [51]:
from kryptools.factor_qs import factor_qs

factor_qs(832283)

1709

## Discrete logarithms

To solve a [discrete log](https://en.wikipedia.org/wiki/Discrete_logarithm) problem use

In [52]:
from random import randint

from kryptools import dlog

p, m, a = [557639, 278819, 2]  # m is the order of a in Z_p
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p) == x

You can also use a specific algorithm:

Exhaustive search:

In [53]:
from kryptools.dlp import dlog_naive

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog(a, b, p, m) == x

[Pollard $\rho$](https://en.wikipedia.org/wiki/Pollard%27s_rho_algorithm) (with Brent's or Floyd's [cycle detection algorithm](https://en.wikipedia.org/wiki/Cycle_detection))

In [54]:
from kryptools.dlp_rho import dlog_rho

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_rho(a, b, p, m, brent=True) == x

Shanks's [baby step/giant step](https://en.wikipedia.org/wiki/Baby-step_giant-step) algorithm

In [55]:
from kryptools.dlp_bsgs import dlog_bsgs

p, m, a = [557639, 278819, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_bsgs(a, b, p, m) == x

[Index calculus](https://en.wikipedia.org/wiki/Index_calculus_algorithm)

In [56]:
from kryptools.dlp_ic import dlog_ic

p, m, a = [24570203447, 12285101723, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_ic(a, b, p, m, verbose=1) == x

Factorbase: bound = 359, size = 72, max_trys = 3000
Success after 69 relations out of 73.


Quadratic sieve

In [57]:
from kryptools.dlp_qs import dlog_qs

p, m, a = [28031135240181527, 14015567620090763, 2]
x = randint(2, m - 1)
b = pow(a, x, p)
assert dlog_qs(a, b, p, m, verbose=1) == x

Factorbase: bound = 1903,  size = 291 + 470 = 761, max_trys = 32530
Done sieving.
Success after 476 relations out of 762.


## Linear Algebra

The basic class for working with [matrices](https://en.wikipedia.org/wiki/Matrix_(mathematics)):

In [58]:
from kryptools import Matrix

M = Matrix([[1, 2, 3], [3, 4, 5]])
N = Matrix([[1], [3], [4]])
M * N

[ 19 ]
[ 35 ]

You can customize the display of a matrix using:

In [59]:
Matrix.print_pre, Matrix.print_sep, Matrix.print_post = "(", " ", ")"

M

(1 2 3)
(3 4 5)

To reset the default use

In [60]:
Matrix.print_pre, Matrix.print_sep, Matrix.print_post = "[ ", ", ", " ]"

Slicing to access (e.g.) rows:

In [61]:
M[:, 0]

[ 1 ]
[ 3 ]

Also works with assignments, e.g., to swap two columns:

In [62]:
M[0, :], M[1, :] = M[1, :], M[0, :]
M

[ 3, 4, 5 ]
[ 1, 2, 3 ]

The coefficients can be from a ring or field, e.g. `Fractions`. This can be done either by using `ring=Fraction` during creation or using the `map` method which applies a given function to all elements.

In [63]:
from fractions import Fraction

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]])
M.map(Fraction)
M

[ 1, 2,  3 ]
[ 4, 5,  6 ]
[ 7, 8, 12 ]

We can compute the [row reduced echelon form](https://en.wikipedia.org/wiki/Row_echelon_form)

In [64]:
M.rref()

[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

the [determinant](https://en.wikipedia.org/wiki/Determinant)

In [65]:
M.det()

-9

or the [inverse](https://en.wikipedia.org/wiki/Invertible_matrix)

In [66]:
Mi = M.inv()
print(Mi)
M * Mi

[ -4/3,    0,  1/3 ]
[  2/3,    1, -2/3 ]
[  1/3, -2/3,  1/3 ]


[ 1, 0, 0 ]
[ 0, 1, 0 ]
[ 0, 0, 1 ]

Same works if the coefficients are from a finite field or ring:

In [67]:
gf = Zmod(11)

M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 12]], ring=gf)
M.det()

2

In [68]:
from kryptools import eye

Mi = M.inv()
assert M * Mi == eye(3, ring=gf)
Mi

[ 6, 0, 4 ]
[ 8, 1, 3 ]
[ 4, 3, 4 ]

Note that `eye` will give you the identity matrix. There is also a function `zeros` to create a zero matrix.

## Lattice

To compute the [Hermite normal form](https://en.wikipedia.org/wiki/Hermite_normal_form) use

In [69]:
from kryptools import hermite_nf

M = Matrix([[1, 2, 3, 1], [4, 5, 6, 4], [7, -8, 0, 7]])
hermite_nf(M)

[ 69, 58, 36 ]
[  0,  1,  0 ]
[  0,  0,  1 ]

To compute the [Gram-Schmidt decomposition](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) use

In [70]:
from kryptools import gram_schmidt

V = Matrix([[5, 8], [0, 1]], ring=Fraction)
Vs, M = gram_schmidt(V)
assert V == Vs * M
Vs

[ 5, 0 ]
[ 0, 1 ]

To compute the [lattice](https://en.wikipedia.org/wiki/Lattice_(group)) determinant/[gram determinant](https://en.wikipedia.org/wiki/Gram_matrix) use

In [71]:
from kryptools import gram_det

gram_det(V)

5.0

To compute the Hadamard ratio use

In [72]:
from kryptools import hadamard_ratio

hadamard_ratio(V)

0.3521856535823236

Approximately solving the closest vector problem using Babai's rounding algorithm

In [73]:
from kryptools import babai_round_cvp

V = Matrix([[5, 8], [0, 1]])
x = Matrix([5.2, 0.4])

a = babai_round_cvp(x, V)
print((a - x).norm())
a

5.215361924162119


[ 0 ]
[ 0 ]

Using Lagrange-Gauß reduction gives a better solution

In [74]:
from kryptools import lagrange_lr

U = lagrange_lr(V)
a = babai_round_cvp(x, U)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

Alternatively we can use Babai's cosest plane algorithm

In [75]:
from kryptools import babai_plane_cvp

V = Matrix([[5, 8], [0, 1]])

a = babai_plane_cvp(x, V)
print((a - x).norm())
a

0.44721359549995804


[ 5 ]
[ 0 ]

[LLL reduction](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm)

In [76]:
from kryptools import lll

V = Matrix([[1, 1, 2], [1, 0, -2], [3, 4, 5]])
U = lll(V)
U

[  0,  1, 2 ]
[ -1,  0, 2 ]
[  1, -1, 1 ]

To get a pseudorandom [unimodular matrix](https://en.wikipedia.org/wiki/Unimodular_matrix) use

In [77]:
from kryptools import random_unimodular_matrix

random_unimodular_matrix(3, max_val=9)

[ 1, -8, -9 ]
[ 0,  3,  4 ]
[ 4,  2,  9 ]

### Showcase: GGH cryptosystem

For example, to generate a key pair for the [GGH cryptosystem](https://en.wikipedia.org/wiki/GGH_encryption_scheme) we start with a nearly orthogonal matrix

In [78]:
from random import randint, seed

from kryptools import eye

seed(0)

n = 4
M = 100
U = 2 * M * n * eye(n) + Matrix([[randint(-M, M) for i in range(n)] for j in range(n)])
U

[ 798,  94,   7, -90 ]
[ -34, 830,  24,   3 ]
[ 100, -23, 822,  -9 ]
[  49, -45,  29, 735 ]

which has a good Hadamrad ratio

In [79]:
hadamard_ratio(U)

0.9962449639096419

Babai's closest pane algorithm will be able to solve the CVP provided the (sup norm of the) error is smaller than:

In [80]:
from math import floor, inf

from kryptools import babai_plane_bnd

babai_plane_bnd(U, inf)

367.329164515002

To obtain the public key we muliply $U$ with a pseudorandom unimodular matrix:

In [81]:
V = U * random_unimodular_matrix(n)
V

[ -1100279,  2579493, -2669106,  2347653 ]
[  -523004,  1175996, -1260068,  1070213 ]
[ -1524616,  3537925, -3692275,  3219934 ]
[   823300, -1867613,  1986482, -1699706 ]

which results in a small Hadamard ratio

In [82]:
hadamard_ratio(V)

0.0002031966189726793

In this case Babai's closest pane algorithm will be able to solve the CVP provided the (sup norm of the) error is smaller than:

In [83]:
babai_plane_bnd(V, inf)

0.015520618275460585

Now we can reagrad the message `FAD!` as a vector of the corresponding ASCII codes $x$ and encrypt the corresponding vector by perturbing the associated lattice vector with a random error $r$:

In [84]:
from math import floor

r_max = floor(babai_plane_bnd(U, inf))
msg = b"FAD!"
assert len(msg) == n

x = Matrix([x for x in msg])
r = Matrix([randint(-r_max, r_max) for _ in range(n)])
y = V * x + r
y

[ -13379202 ]
[ -10538159 ]
[ -21574537 ]
[  15226425 ]

Using the private basis $U$ we can decrypt (we change from integers to `Fraction` for the matrix inversion to avoid rounding errors)

In [85]:
V.map(Fraction)
V.inv() * babai_round_cvp(y, U) == x

True

Using the public basis $V$ decryption fails

In [86]:
V.map(Fraction)
V.inv() * babai_round_cvp(y, V) == x

False

Since our parameters are too small, the system can be broken using LLL reduction:

In [87]:
UU = lll(V)
V.inv() * babai_plane_cvp(y, UU) == x

True

## Polynomials

A basic class for [polynomials](https://en.wikipedia.org/wiki/Polynomial), entered as a list of coefficients

In [88]:
from kryptools import Poly

p = Poly([2, 0, 2])
q = Poly([1, 2, 3, 4])

p, q

(2 x^2 + 2, 4 x^3 + 3 x^2 + 2 x + 1)

The usual arithmetic operations work as expected

In [89]:
p * q

8 x^5 + 6 x^4 + 12 x^3 + 8 x^2 + 4 x + 2

[Polynomial division](https://en.wikipedia.org/wiki/Polynomial_long_division) with remainder

In [90]:
d, m = q.divmod(p)

assert d * p + m == q
d, m

(2.0 x + 1.5, -2.0 x - 2.0)

Alternatively

In [91]:
q // p, q % p

(2.0 x + 1.5, -2.0 x - 2.0)

If you prefer a different ordering of the coefficients, you can use

In [92]:
Poly.print_reversed = False
p

2 + 2 x^2

In [93]:
Poly.print_reversed = True

To avoid floats, we can work with fractions. Either by using `ring=Fraction` during creation or by applying `Fraction` to all coefficients (in place):

In [94]:
p.map(Fraction)
q.map(Fraction)

q.divmod(p)

(2 x + 3/2, -2 x - 2)

If you want the reduction of $q$ modulo $p$ to be made in place use:

In [95]:
q.mod(p)
q

-2 x - 2

The maximum of the absolute value of all coefficients can be obtained with

In [96]:
q.max()

2

and the sum of the absolute value of all coefficients with

In [97]:
q.sum()

4

### Showcase: AES

Computations in a polynomial ring modulo a given polynomial, for example the [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) Polynomial $x^8+x^4+x^3+x+1$, can be done as follows:

In [98]:
from kryptools import Poly, Zmod

Z_2 = Zmod(2)
aes = [1, 1, 0, 1, 1, 0, 0, 0, 1]  # 1 + x + x^3 + x^4 + x^8


def PolyAES(c: list) -> "Poly":
    return Poly(c, ring=Z_2, modulus=aes)


p = PolyAES([1, 0, 1])
q = PolyAES([0, 1, 0, 0, 1, 1, 1, 1])

p * q

x^4 + x^2 + x + 1

For AES, bytes are identified with polynomials (in the Galois field $\mathbb{Z}_2[x]_{x^8+x^4+x+1}$) by identifying the individual bits with the coefficients of the polynomial. The nonlinear operation used in AES is given by computing the inverse in the above field.

In [99]:
def byte2poly(x: int, n: int = 8) -> "Poly":
    "Convert an integer to a polynomial."
    return PolyAES(list(reversed([int(d) for d in str(format(x, "0" + str(n) + "b"))])))


def poly2byte(p: "Poly") -> int:
    "Convert a polynomial to the corresponding integer."
    s = 0
    for i in reversed(p.coeff):
        s = s * 2 + int(i)
    return s


poly2byte(byte2poly(2).inv())

141

For example, a substitution table for the [AES S-box](https://en.wikipedia.org/wiki/Rijndael_S-box) can be computed as follows:

In [100]:
from kryptools import Matrix, zeros


def rotate(l: list, n: int) -> list:
    "Rotate a list cyclically n places to the left."
    n %= len(l)
    return l[n:] + l[:n]


A = Matrix([rotate([1, 0, 0, 0, 1, 1, 1, 1], -n) for n in range(8)])
b = Matrix([1, 1, 0, 0, 0, 1, 1, 0])


def aes_sbox(x: int | Poly) -> int:
    "AES S-box."
    if isinstance(x, int):
        x = byte2poly(x)
    if x:
        res = x.inv().coeff
        res += [0] * (8 - len(res))  # pad the result with zeros
    else:
        res = [0] * 8
    res = A * Matrix(res) + b
    s = 0
    for i in reversed(res):
        s = s * 2 + int(i)
    return s


Matrix(
    [[hex(aes_sbox(i + 16 * j)) for i in range(16)] for j in range(16)]
)  # Matrix(...) is just to get a nicer output

[ 0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30,  0x1, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76 ]
[ 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0 ]
[ 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15 ]
[  0x4, 0xc7, 0x23, 0xc3, 0x18, 0x96,  0x5, 0x9a,  0x7, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75 ]
[  0x9, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84 ]
[ 0x53, 0xd1,  0x0, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf ]
[ 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9,  0x2, 0x7f, 0x50, 0x3c, 0x9f, 0xa8 ]
[ 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2 ]
[ 0xcd,  0xc, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73 ]
[ 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e,  0xb, 0xdb ]
[ 0xe0, 0x

Alternatively you can use the built-in Galois field:

In [101]:
from kryptools import GF2_aes

x = 3
assert int(GF2_aes(x) ** -1) == poly2byte(byte2poly(x).inv())
assert int(GF2_aes(x).sbox()) == aes_sbox(x)

The [MixColumn](https://en.wikipedia.org/wiki/Rijndael_MixColumns) operation can be expressed as polynomial multiplication of polynomials with coefficients in $\mathrm{GF}(8)$ modulo $x^4+1$. Each column of the state matrix is regarded as a polynomial and multiplied with $3x^3+x^2+x+2$. The inverse is given by

In [102]:
Poly([2, 1, 1, 3], modulus=[1, 0, 0, 0, 1], ring=gf).inv()

8 x^3 + 8 x^2 + 2 x + 6

To verify that a given poynomial is irreducible over a Galois field we need the gcd

In [103]:
a = Poly([1, -2, 0, 1])
b = Poly([-1, 0, 1])
a.gcd(b)

- x + 1

and a variant of [Rabin's irreducibility test](https://en.wikipedia.org/wiki/Factorization_of_polynomials_over_finite_fields#Rabin.27s_test_of_irreducibility)

In [104]:
def rabin_test(a: Poly, verbose: bool = False) -> bool:
    """Determine if a polynomial is irreducible over a Galois field using a Rabin test."""
    gf = a[0].ring
    p = gf.n
    x = Poly([0, 1], ring=gf)  # x
    xp = Poly(
        p * [0] + [1], ring=gf, modulus=a.coeff
    )  # x^p modulo a to keep the polynomials small
    for k in range(1, a.degree() // 2 + 1):
        if k == 1:
            b = xp
        else:
            b **= p  # we compute x^{p^k} mod a recursively
        g = a.gcd(b - x)  # gcd with x^{p^k} - x
        if g.degree():
            if verbose:
                print(f"Factor found with k={k}: {g}")
            return False
    return True


a = Poly([1, 0, 1], ring=Z_2)

rabin_test(a, verbose=True)

Factor found with k=1: x + 1


False

Now we can show that the AES polynomial $x^8+x^4+x^3+x+1$ is irreducible:

In [105]:
a = Poly(aes, ring=Z_2)

rabin_test(a)

True

Similarly for the Ghash polynomial $x^{128} + x^7 + x^2 + x + 1$:

In [106]:
a = Poly([1, 1, 1, 0, 0, 0, 0, 1] + 120 * [0] + [1], ring=Z_2)

rabin_test(a)

True

Note that the number of irreducible monic polynomials of degree $k$ in the field $\mathbb{Z}_p$ is given by the [necklace polynomial](https://en.wikipedia.org/wiki/Necklace_polynomial).
For $\mathbb{Z}_2$ there are for example $30$ irreducible monic polynomials of degree $8$.

In [107]:
from kryptools import divisors, moebius_mu


def M(p: int, k: int) -> int:
    """Moreau's necklace-counting function."""
    return sum([moebius_mu(d) * p ** (k // d) for d in divisors(k)]) // k


M(2, 8)

30

We can list them as follows:

In [108]:
for i in range(256):
    a = Poly(
        list(reversed([int(d) for d in str(format(256 + i, "0" + str(9) + "b"))])),
        ring=Z_2,
    )
    if rabin_test(a):
        print(a)

x^8 + x^4 + x^3 + x + 1
x^8 + x^4 + x^3 + x^2 + 1
x^8 + x^5 + x^3 + x + 1
x^8 + x^5 + x^3 + x^2 + 1
x^8 + x^5 + x^4 + x^3 + 1
x^8 + x^5 + x^4 + x^3 + x^2 + x + 1
x^8 + x^6 + x^3 + x^2 + 1
x^8 + x^6 + x^4 + x^3 + x^2 + x + 1
x^8 + x^6 + x^5 + x + 1
x^8 + x^6 + x^5 + x^2 + 1
x^8 + x^6 + x^5 + x^3 + 1
x^8 + x^6 + x^5 + x^4 + 1
x^8 + x^6 + x^5 + x^4 + x^2 + x + 1
x^8 + x^6 + x^5 + x^4 + x^3 + x + 1
x^8 + x^7 + x^2 + x + 1
x^8 + x^7 + x^3 + x + 1
x^8 + x^7 + x^3 + x^2 + 1
x^8 + x^7 + x^4 + x^3 + x^2 + x + 1
x^8 + x^7 + x^5 + x + 1
x^8 + x^7 + x^5 + x^3 + 1
x^8 + x^7 + x^5 + x^4 + 1
x^8 + x^7 + x^5 + x^4 + x^3 + x^2 + 1
x^8 + x^7 + x^6 + x + 1
x^8 + x^7 + x^6 + x^3 + x^2 + x + 1
x^8 + x^7 + x^6 + x^4 + x^2 + x + 1
x^8 + x^7 + x^6 + x^4 + x^3 + x^2 + 1
x^8 + x^7 + x^6 + x^5 + x^2 + x + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x^2 + 1
x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + 1


### Showcase: NTRU

Creating a public/private key pair suitable for a baby version of [NTRU](https://en.wikipedia.org/wiki/NTRU)

In [109]:
from copy import copy
from random import sample, seed

seed(0)


def ntru(n: int) -> list:
    return [-1] + [0] * (n - 1) + [1]  # x^n - 1


def random_ternery_poly(n: int, dp: int, dm: int, modulus: list = None) -> "Poly":
    res = [0] * n
    smpl = sample(range(n), dp + dm)
    for i in smpl[:dp]:
        res[i] = 1
    for i in smpl[dp:]:
        res[i] = -1
    return Poly(res, modulus=modulus)


n = 5
q = 41
p = 3
d = 2

gf_q = Zmod(q)
gf_p = Zmod(p)


found = False
while not found:
    f = random_ternery_poly(n, 3, 2, modulus=ntru(n))

    fp = copy(f)
    fp.map(gf_p)
    fq = copy(f)
    fq.map(gf_q)

    try:
        Fq = fq.inv()
        Fp = fp.inv()
        found = True
    except:
        pass

g = random_ternery_poly(n, 2, 2, modulus=ntru(n))
h = Fq * g
h
print("Private key:")
print(f"f = {f}, F_p = {Fp}")

print("Public key:")
print(f"(F_q = {Fp}, g = {g})")
print(f"h = {h}")

Private key:
f = x^4 + x^3 - x^2 - x + 1, F_p = - x^2 + 2
Public key:
(F_q = - x^2 + 2, g = - x^4 + x^3 + x^2 - x)
h = 21 x^2 - x + 21


### Showcase: Kyber

Creating a public/private key pair suitable for a baby version of [Kyber](https://en.wikipedia.org/wiki/Kyber)

In [110]:
from random import randint, seed

try:
    from random import binomialvariate
except:  # for versions older then 3.12
    from random import random

    def binomialvariate(n: int = 1, p: float = 0.5) -> int:
        "Binomial random variable"
        return sum(random() < p for i in range(n))


seed(0)

p = 97
n = 4
k = 2
gf = Zmod(p)


def kyber(n: int) -> list:
    return [1] + [0] * (n - 1) + [1]  # x^n + 1


def PolyKyber(c: list) -> "Poly":
    return Poly(c, ring=gf, modulus=kyber(n))


A = Matrix(
    [
        [PolyKyber([randint(0, p - 1) for _ in range(n)]) for i in range(k)]
        for j in range(k)
    ]
)
s = Matrix([PolyKyber([randint(-1, 1) for _ in range(n)]) for i in range(k)])
q = p // (4 * k * n)
e = Matrix(
    [PolyKyber([binomialvariate(2 * q) - q for _ in range(n)]) for i in range(k)]
)
b = A * s + e

print("Private key:")
print(s)

print("Public key:")
print(A)
print(b)

Private key:
[  x^3 + x + 96 ]
[ - x^2 + x + 1 ]
Public key:
[  33 x^3 + 5 x^2 + 53 x + 49, 38 x^3 + 51 x^2 + 62 x + 65 ]
[ 27 x^3 + 74 x^2 + 45 x + 61, 17 x^3 + 36 x^2 + 17 x + 64 ]
[ 47 x^3 + 63 x^2 + 60 x + 38 ]
[ 47 x^3 + 30 x^2 + 42 x + 47 ]


Computations can be speed up using the fact that the $x^{256}+1$ can be factored as $\prod_{j=0}^{128} (x^2 + 17^{2j+1})$ in $\mathbb{Z}_{3329}$:

In [111]:
from math import prod

p = 3329
gf = Zmod(p)

m = [Poly([pow(17, 2 * j + 1, p), 0, 1], ring=gf) for j in range(128)]
M = prod(m)
M

x^256 + 1

Note that the factors $m_j(x)=x^2 + 17^{2j+1}$ are irreducible since $17$ is not a quadratic residue:

In [112]:
from kryptools import legendre_symbol

legendre_symbol(17, p)

-1

Hence a given polynomial $a(x)$ can be replaced by its remainders modulo $m_j(x)$ and all computations can be done with the remainders. The resulting polynomial can be constructed from the remainders using the Chinese Remainder Theorem. This is known as Number-Theoretic-Transform in this context.

In [113]:
def ntt(a: list["Poly"], m: list["Poly"]) -> "Poly":
    """Solve given linear polynomial congruences x % m[j] == a[j] using the Chinese Remainder Theorem."""
    l = len(a)
    assert len(m) == l, "The lists of numbers and modules must have equal length."
    M = prod(m)
    Mi = [M.divmod(m[i])[0] for i in range(l)]
    for i in range(l):
        Mi[i].modulus = M.coeff
    MNi = [Mi[i] * Mi[i].inv(m[i]) for i in range(l)]
    return sum([(a[i] * MNi[i]).divmod(M)[1] for i in range(l)])


a = Poly([1, 2, 3, 4, 5, 6], ring=gf)
aj = [a.divmod(m[j])[1] for j in range(128)]

ntt(aj, m)

6 x^5 + 5 x^4 + 4 x^3 + 3 x^2 + 2 x + 1

## Eliptic curves

To declare an [elliptic curve](https://en.wikipedia.org/wiki/Elliptic_curve) in Weierstrass normal form use

In [114]:
from kryptools import EC_Weierstrass

ec = EC_Weierstrass(239, 3, 1)
ec.info()
print("Order:", ec.order())

Weierstrass curve y^2 = x^3 + 3 x + 1 over Z_239.
Order: 258


The order is computed using the Shanks-Maestre algorithm.

Basic arithmetic

In [115]:
P = ec.random()
Q = ec.random()
O = ec.inf()  # point at infinity
P + Q

(113, 91)

To check if a point is on the curve use

In [116]:
P in ec

True

To compute the order of a point use

In [117]:
assert Q.order() * Q == O
Q.order()

129

The $n$'th division polynomial $\Psi_n(x,y)$ will anihilate all points whose order is a multiple of $n$:

In [118]:
P.psi(P.order())

0

To compute a discrete log (Pohlig-Hellman reduction and Shank's baby step/giant step algorithm) use

In [119]:
k = 12
R = k * Q
assert R.dlog(Q) == k % Q.order()

### Showcase: Bitcoin curve

Setting the NIST curve [`secp256k1`](https://en.bitcoin.it/wiki/Secp256k1)

In [120]:
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1
m = 115792089237316195423570985008687907852837564279074904382605163141518161494337  # order of the curve
ec = EC_Weierstrass(p, 0, 7, m)
ec.hex = True  # display points in hex form
ec.short = True  # display points in short form
G = ec(
    "02 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798"
)  # generator

Generate a (pseudorandom) key pair

In [121]:
secret_key = randint(2, m - 1)
public_key = secret_key * G
secret_key, public_key

(23529166972251975810298992610444093284962271729068871731505428628592805361023,
 0x03449ef2ed30a8deb96e584a08c329adbf1be87ce40f1a0e7b4e86178682c41a9c)

## DES

The DES module implements a basic DES-type Feistel network plus the corresponding key-schedule. Specifying the corresponding permutations and sboxes, one can get instances of S-DES and DES.

For SDES the two classes are `SDESKeySchedule` and `SDESCipher`. The first class is used to compute the round keys. Internally everything is represented as a list of the individual bits. This is the state to begin with:

In [122]:
from kryptools import SDESKeySchedule

ks = SDESKeySchedule(34)
ks

[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]

If you prefer the output as a hex number, you can use the following option:

In [123]:
SDESKeySchedule.print_hex = True
print(ks)
SDESKeySchedule.print_hex = False

104


Using the `next()` method you can generate the keys one after the other

In [124]:
ks.next()

[0, 0, 1, 0, 0, 0, 0, 0]

You can also generate all round keys (in the case of SDES there are just two) at once. But you might want to reset the state in case you already generated some keys.

In [125]:
ks.reset()
keys = ks.gen_keys()
keys

[[0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1, 0]]

There is also a handy function for converting the list of bits back to an integer:

In [126]:
from kryptools import list2int

list2int(keys[0])

32

Of course there is also a function which converts an integer to a list of bits with fixed length given by the second argument:

In [140]:
from kryptools import int2list

int2list(32, 8)

[0, 0, 1, 0, 0, 0, 0, 0]

Next, there is the `SDESCipher` class. It gives you access to all individual operations

In [149]:
from kryptools import SDESCipher

x = b"\x01"  # the message to be encrypted
sdes = SDESCipher()
sdes(x)  # set up the cipher

print(sdes)
sdes.ip()  # apply the initial permutation
print(sdes)

sdes.round(keys[0])  # apply one round with the given round key
print(sdes)

sdes.round(keys[1], last=True)  # The last round will not switch the two halfes
print(sdes)

sdes.fp()  # apply the final permutation
# print(sdes)

int(sdes)

00000001
00000100
01001110
00111110


185

Of course you can also encrypt/decrypt individual blocks. The result is a single byte and you can use the following trick to get the result as an integer:

In [153]:
sdes = SDESCipher(34)
sdes.encrypt(x)[0]

185

Note that it is easy to change the individual components of the Feistel cipher. For example if you change the schedule of left shifts for the key schedule for S-DES from `ROT = (1, 2)` to (say) `ROT = (1, 2, 2, 1)`, you will get a variant of S-DES with 4 rounds. This is conveniant if you want to apply linear or differential cryptoanalysis (which is kind of pointless for a Feistel network with only two rounds).

In [154]:
sdes.key_schedule.ROT = (1, 2, 2, 1)
sdes.set_key(34)  # to use the new key schedule
sdes.encrypt(x)[0]

107

Of course the same works with DES. Here are some test vectors from  the [Handbok of Applied Cryptography](https://cacr.uwaterloo.ca/hac/) (Example 7.86):

In [130]:
from kryptools import DESCipher

key = 0x0123456789ABCDEF.to_bytes(8, "big")
x = 0x4E6F772069732074.to_bytes(8, "big")
y = 0x3FA40E8A984D4815.to_bytes(8, "big")

des = DESCipher(key)
des(x).encrypt()
print(des)
assert bytes(des) == y
assert des.decrypt() == x

3fa40e8a984d4815


Here are the sample round outputs from [NBS SP500-20](https://doi.org/10.6028/NBS.SP.500-20e1980) (Figure 4 on p9):

In [131]:
key = 0x10316E028C8F3B4A.to_bytes(8, "big")
x = 0x0000000000000000.to_bytes(8, "big")
y = 0x82DCBAFBDEAB6602.to_bytes(8, "big")


des = DESCipher(key)
des(x)
for i in range(16):
    fbox = list2int(des.f_box(des.right(), des.keys[i]))
    roundkey = list2int(des.keys[i])
    des.round(des.keys[i], last=(i == 15))
    res = str(des)
    print(
        f"round {i:02d}: {res[:8]} {res[8:]}  (key= {roundkey:012x}, fbox= {fbox:08x})"
    )
des.fp()
assert bytes(des) == y
des

round 00: 00000000 47092b5b  (key= 4220438e6f23, fbox= 47092b5b)
round 01: 47092b5b 53f372af  (key= 182235355963, fbox= 53f372af)
round 02: 53f372af 9f1d158b  (key= 851408c68876, fbox= d8143ed0)
round 03: 9f1d158b 8109cbee  (key= 4202e4c5afdc, fbox= d2fab941)
round 04: 8109cbee 60448698  (key= 98d8003996d9, fbox= ff599313)
round 05: 60448698 29ebb1a4  (key= 00236a5bd427, fbox= a8e27a4a)
round 06: 29ebb1a4 620cc3a3  (key= a054050e6dac, fbox= 0248453b)
round 07: 620cc3a3 deeb3d8a  (key= 410b40a879d5, fbox= f7008c2e)
round 08: deeb3d8a a1a0354d  (key= 620078ef2871, fbox= c3acf6ee)
round 09: a1a0354d 9f0303dc  (key= 8cc004a3eb5a, fbox= 41e83e56)
round 10: 9f0303dc fd898ee8  (key= 020b1a359716, fbox= 5c29bba5)
round 11: fd898ee8 2d1ae1dd  (key= 2c3021dd04e6, fbox= b219e201)
round 12: 2d1ae1dd cbc829fa  (key= 830c484ceacd, fbox= 3641a712)
round 13: cbc829fa b367dec9  (key= 48629032f4dd, fbox= 9e7d3f14)
round 14: b367dec9 3f6c3efd  (key= 149d08ab95a3, fbox= f4a41707)
round 15: 5a1e5228 3f6c3e

82dcbafbdeab6602

## AES

The AES module implements two classes: `AESKeySchedule` and `AESCipher`. The first class is used to compute the round keys. For example, here are the first two rounds of the key schedule with details:

In [132]:
from kryptools import AESKeySchedule

key = 0x2B7E151628AED2A6ABF7158809CF4F3C.to_bytes(16, "big")
ks = AESKeySchedule(key)
ks.showtmp = True

print(ks)
print("------------------")
for _ in range(2):
    print(f"----Round {ks.round+1}-------")
    ks.next()
    print(ks)

[ 2b, 28, ab, 09 ]
[ 7e, ae, f7, cf ]
[ 15, d2, 15, 4f ]
[ 16, a6, 88, 3c ]
------------------
----Round 1-------
Start    :[09, cf, 4f, 3c]
RotWord  :[cf, 4f, 3c, 09]
SubWord  :[8a, 84, eb, 01]
AddConst :[8b, 84, eb, 01]
[ a0, 88, 23, 2a ]
[ fa, 54, a3, 6c ]
[ fe, 2c, 39, 76 ]
[ 17, b1, 39, 05 ]
----Round 2-------
Start    :[2a, 6c, 76, 05]
RotWord  :[6c, 76, 05, 2a]
SubWord  :[50, 38, 6b, e5]
AddConst :[52, 38, 6b, e5]
[ f2, 7a, 59, 73 ]
[ c2, 96, 35, 59 ]
[ 95, b9, 80, f6 ]
[ f2, 43, 7a, 7f ]


Using the `AEScipher` class you can watch how the state changes during individual rounds:

In [133]:
from kryptools import AESCipher

verbose = True
key = 0x2B7E151628AED2A6ABF7158809CF4F3C.to_bytes(16, "big")
x = 0x3243F6A8885A308D313198A2E0370734.to_bytes(16, "big")

state = AESCipher(key)
state(x)  # initialize the state
print(state)
print("------------------")
print("----AddKey--------")
state.add_key(state.keys[0])
print(state)
for r in range(1, 2):
    print(f"----Round {r}-------")
    if verbose:
        print("----SubBytes------")
    state.sub_bytes()
    if verbose:
        print(state)
        print("----ShiftRows-----")
    state.shift_rows()
    if verbose:
        print(state)
    if r != 10:
        if verbose:
            print("----MixColumns----")
        state.mix_columns()
        if verbose:
            print(state)
    if verbose:
        print("----AddKey--------")
    state.add_key(state.keys[r])
    print(state)

[ 32, 88, 31, e0 ]
[ 43, 5a, 31, 37 ]
[ f6, 30, 98, 07 ]
[ a8, 8d, a2, 34 ]
------------------
----AddKey--------
[ 19, a0, 9a, e9 ]
[ 3d, f4, c6, f8 ]
[ e3, e2, 8d, 48 ]
[ be, 2b, 2a, 08 ]
----Round 1-------
----SubBytes------
[ d4, e0, b8, 1e ]
[ 27, bf, b4, 41 ]
[ 11, 98, 5d, 52 ]
[ ae, f1, e5, 30 ]
----ShiftRows-----
[ d4, e0, b8, 1e ]
[ bf, b4, 41, 27 ]
[ 5d, 52, 11, 98 ]
[ 30, ae, f1, e5 ]
----MixColumns----
[ 04, e0, 48, 28 ]
[ 66, cb, f8, 06 ]
[ 81, 19, d3, 26 ]
[ e5, 9a, 7a, 4c ]
----AddKey--------
[ a4, 68, 6b, 02 ]
[ 9c, 9f, 5b, 6a ]
[ 7f, 35, ea, 50 ]
[ f2, 2b, 43, 49 ]


## Block Ciphers

The `BlockCipher` class provides various mode of operations for a block cipher. One way of using it is to provide the encryption/decryption function for one block. An instance gets initialized with the secret key and a mode of operation (`ecb`, `cbc`, etc). If the chosen mode requires padding, PKCS#7 will be used. Also a random IV (initial vector) will be choosen automatically and added to the cipher text as the first block. Accordingly, decryption expects the IV as first block for such modes.

In [134]:
from kryptools import BlockCipher, bytexor


class XORBlockCipher(BlockCipher):
    blocksize = 2

    def encrypt_block(self, b: bytes):
        return bytexor(b, self.key)

    def decrypt_block(self, b: bytes):
        return bytexor(b, self.key)


xor2 = XORBlockCipher(b"az", mode="cbc")
text = b"test1"
ctext = xor2.encrypt(text)
assert text == xor2.decrypt(ctext)
ctext

b'\x08\xa3\x1d\xbc\x0f\xb2_\xc9'

Another way of using it is to provide a class for the cipher which supports an encrypt/decrypt method.

In [135]:
class CaesarCipher:
    "Caesar cipher"

    def __init__(self, key: bytes):
        self.key = key

    def encrypt(self, text: bytes):
        "Encrypt one block."
        ctext = bytearray(b"\x00" * len(self.key))
        for i in range(len(self.key)):
            ctext[i] = (text[i] + self.key[i]) % 256
        return bytes(ctext)

    def decrypt(self, ctext: bytes):
        "Decrypt one block."
        text = bytearray(b"\x00" * len(self.key))
        for i in range(len(self.key)):
            text[i] = (ctext[i] - self.key[i]) % 256
        return bytes(text)


class CaesarBlockCipher(BlockCipher):
    blocksize = 4

    def set_key(self, key: bytes):
        if len(key) != self.__class__.blocksize:
            raise ValueError(f"Key must be {self.__class__.blocksize} bytes.")
        self.key = key
        self.cipher = CaesarCipher(key)


ceasar = CaesarBlockCipher(key=b"abcd", mode="cbc")
text = b"test123"
ctext = ceasar.encrypt(text)
assert text == ceasar.decrypt(ctext)
ctext

b'\xab_\xe8\xf1@\x9c\xfe\xe9\xd2\x100L'

You can also compute a cbc mac

In [136]:
ceasar.mac_cbc(text)

b'EWH='

or a CMAC (the blocksize (in bytes) must be 1,2,4,8 or 16)

In [137]:
ceasar.mac_cmac(text)

b'\xc2\xde\xcb\xa9'

Of course you can use the classes for DES or AES from above.

In [138]:
from kryptools import AESCipher, BlockCipher


class AESBlockCipher(BlockCipher):
    blocksize = 16

    def set_key(self, key: bytes):
        if not isinstance(key, bytes) or len(key) != self.__class__.blocksize:
            raise ValueError(f"Key must be {self.__class__.blocksize} bytes.")
        self.key = key
        self.cipher = AESCipher(key)

For example you can use Galois counter mode with associated authenticated data:

In [139]:
key = bytes.fromhex("36f18357be4dbd77f050515c73fcf9f2")
text = b"Test message"
aad = b"ab"

aes = AESBlockCipher(key)
ctext = aes.encrypt_gcm(text, aad=aad)
assert aes.decrypt_gcm(ctext, aad=aad) == text
ctext

b"\xf1\xf5A\xac^OuvE2\xbe\x93\xcf\xdb\xe4\x83M\xcc\xaa\xc2\xf0\xebw'\xb1Z\x8dN\xdb7v\xe1\x99\t\n\\\xb9\x12(O"